In [23]:
import pandas as pd
import numpy as np

# 1. 데이터 로드 (로컬 파일 경로 수정)
file_path = "D:/코딩/Python/AI/AI_Project/Deep_Learning/Dummy_02.csv"  # 로컬 경로로 변경
df = pd.read_csv(file_path)


In [24]:
import pandas as pd
import numpy as np

# 이상 데이터 생성 함수 (event_time 증가 추가)
def create_anomalies_with_time_shift(df, seq_length=10):
    anomaly_df = df.copy()
    
    # event_time을 datetime 형식으로 변환 (문자열이면 변환)
    if df['event_time'].dtype == 'object':  
         anomaly_df['event_time'] = pd.to_datetime(anomaly_df['event_time'])
    
    grouped = anomaly_df.groupby('epc_code')
    anomaly_samples = []
    
    for _, group in grouped:
        if len(group) >= seq_length:
            group = group.copy()
            
            # (1) 허브 이동 순서 랜덤 변경
            shuffled_hub = group['hub_type'].sample(frac=1, random_state=42).values
            group['hub_type'] = shuffled_hub

            # (2) 이벤트 순서 랜덤 변경
            shuffled_event = group['event_type'].sample(frac=1, random_state=42).values
            group['event_type'] = shuffled_event

            # (3) 특정 샘플의 `event_time`을 무작위로 증가 (월, 일, 시간, 분, 초)
            time_shift_idx = np.random.choice(len(group), size=1, replace=False)
            shift_amount = np.random.choice([
                pd.DateOffset(months=1),  # 1개월 증가
                pd.DateOffset(days=10),   # 10일 증가
                pd.DateOffset(hours=5),   # 5시간 증가
                pd.DateOffset(minutes=30), # 30분 증가
                pd.DateOffset(seconds=55) # 55초 증가
            ])
            
            group.loc[group.index[time_shift_idx], 'event_time'] += shift_amount
            
            anomaly_samples.append(group)
    
    anomaly_df = pd.concat(anomaly_samples, ignore_index=True)
    
    # 라벨 추가 (정상 데이터: 0, 이상 데이터: 1)
    df['label'] = 0  # 원래 데이터는 정상 (0)
    anomaly_df['label'] = 1  # 이상 데이터 (1)
    
    return df, anomaly_df

# 이상 데이터 생성
df_normal, df_anomalies = create_anomalies_with_time_shift(df)

# 최종 데이터셋 병합
df_final = pd.concat([df_normal, df_anomalies], ignore_index=True)

# CSV 파일로 저장
output_file_path = "D:/코딩/Python/AI/AI_Project/Deep_Learning/학습데이터.csv"
df_final.to_csv(output_file_path, index=False)

print(f"이상 데이터가 생성되어 저장되었습니다: {output_file_path}")


이상 데이터가 생성되어 저장되었습니다: D:/코딩/Python/AI/AI_Project/Deep_Learning/학습데이터.csv
